# **Enriquecimiento de anotaciones para Proteínas Moonlighting usando UniProt y BioPython**
Este notebook detalla el proceso de enriquecimiento de dos conjuntos de datos de proteínas moonlighting — MoonProt y MoonDB — utilizando información de UniProt y herramientas de BioPython. El objetivo es ampliar estos conjuntos de datos con información relevante y actualizada.


## Montaje de Google Drive e instalación de BioPython
Montamos Google Drive para acceder a los datos almacenados e instalamos BioPython en el entorno para su uso en el análisis.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip3 install biopython

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preparación de los datos para MoonProt
Cargamos y preparamos los identificadores UniProt del conjunto de datos MoonProt para su posterior análisis.

In [ ]:
import pandas as pd
import re

# Cargamos los datos de MoonProt.
data_moonprot = pd.read_csv('/content/drive/My Drive/raw_moonprot.csv', header=1, encoding='ISO-8859-1')

# Limipamos los identificadores UniProt de MoonProt.
data_moonprot['uniprot_id'] = data_moonprot['uniprot_id'].str.replace(r'\(.*?\)', '', regex=True)
data_moonprot['uniprot_id'] = data_moonprot['uniprot_id'].str.replace(r'[\(\)]', '', regex=True)
data_moonprot['uniprot_id'] = data_moonprot['uniprot_id'].str.strip()

uniprot_ids = data_moonprot['uniprot_id'].tolist()

## Extracción de datos de UniProt para MoonProt
Utilizamos BioPython para extraer datos detallados de cada proteína usando sus identificadores UniProt.


In [ ]:
from Bio import ExPASy, SwissProt   # ExPASy: realiza consultas en línea a la base de datos SwissProt/UniProt.
                                    # SwissProt: lee y analiza los registros de las proteínas obtenidos.

# Inicializamos diccionarios para almacenar la información recopilada para cada
# proteína.
pdb_ids = {}
gene_names = {}
protein_names = {}
amino_acid_sequences = {}
organism_names = {}
status_info = {}
protein_existence = {}
go_bp_terms = {}  # Términos Biological Process.
go_cc_terms = {}  # Términos Cellular Component.
go_mf_terms = {}  # Términos Molecular Function.

# Definimos una función para unir datos y manejar valores que podrían no ser cadenas.
def safe_str_join(data, separator="; "):
  if isinstance(data, list):
    return separator.join(str(item) for item in data)
  elif isinstance(data, dict):
    return separator.join(f"{key}: {value}" for key, value in data.items())
  elif data is None:
    return "N/A"
  return str(data)

# Iteramos sobre cada identificador UniProt para realizar consultas y recopilar información.
for uniprot_id in uniprot_ids:
  try:
    handle = ExPASy.get_sprot_raw(uniprot_id)
    record = SwissProt.read(handle)

    # Almacenamos la información específica obtenida de cada registro.
    pdb_entries = [safe_str_join(ref[1]) for ref in record.cross_references if ref[0] == 'PDB']
    pdb_ids[uniprot_id] = safe_str_join(pdb_entries)
    gene_names[uniprot_id] = safe_str_join(record.gene_name, ", ")
    protein_names[uniprot_id] = safe_str_join(record.description.split("=")[1].split(";")[0])
    amino_acid_sequences[uniprot_id] = safe_str_join(record.sequence)
    organism_names[uniprot_id] = safe_str_join(record.organism)
    status_info[uniprot_id] = "Reviewed" if record.data_class == "Reviewed" else "Unreviewed"
    protein_existence[uniprot_id] = safe_str_join(record.protein_existence)

    # Procesamos los términos GO asociados a cada proteína, clasificándolos según su categoría.
    go_terms_processing = lambda ref, code: [f"{r[1]} - {r[2][2:]}" for r in ref if r[0] == "GO" and r[2].startswith(code)]
    go_bp_terms[uniprot_id] = safe_str_join(go_terms_processing(record.cross_references, "P:"))
    go_cc_terms[uniprot_id] = safe_str_join(go_terms_processing(record.cross_references, "C:"))
    go_mf_terms[uniprot_id] = safe_str_join(go_terms_processing(record.cross_references, "F:"))

  except Exception as e:
    print(f"Error al procesar el identificador {uniprot_id}: {e}")
    pdb_ids[uniprot_id] = gene_names[uniprot_id] = protein_names[uniprot_id] = amino_acid_sequences[uniprot_id] = organism_names[uniprot_id] = go_bp_terms[uniprot_id] = go_cc_terms[uniprot_id] = go_mf_terms[uniprot_id] = status_info[uniprot_id] = protein_existence[uniprot_id] = "Error al recuperar datos"

# Actualizamos el DataFrame con los datos recopilados.
data_moonprot['PDB ID'] = data_moonprot['uniprot_id'].map(pdb_ids)
data_moonprot['Gene Name'] = data_moonprot['uniprot_id'].map(gene_names)
data_moonprot['Protein Name'] = data_moonprot['uniprot_id'].map(protein_names)
data_moonprot['Amino Acid Sequence'] = data_moonprot['uniprot_id'].map(amino_acid_sequences)
data_moonprot['Organism Name'] = data_moonprot['uniprot_id'].map(organism_names)
data_moonprot['Status'] = data_moonprot['uniprot_id'].map(status_info)
data_moonprot['Protein Existence'] = data_moonprot['uniprot_id'].map(protein_existence)
data_moonprot['GO BP Terms'] = data_moonprot['uniprot_id'].map(go_bp_terms)
data_moonprot['GO CC Terms'] = data_moonprot['uniprot_id'].map(go_cc_terms)
data_moonprot['GO MF Terms'] = data_moonprot['uniprot_id'].map(go_mf_terms)

data_moonprot_final = data_moonprot.rename(columns={'uniprot_id': 'UniProt IDs'})[
    ['UniProt IDs', 'PDB ID', 'Gene Name', 'Protein Name', 'Amino Acid Sequence', 'Organism Name', 'Status', 'Protein Existence', 'GO BP Terms', 'GO CC Terms', 'GO MF Terms']]

# Visualizamos las primeras filas del DataFrame obtenido.
data_moonprot_final.head()

Error al procesar el identificador A0A1F1EC06: No SwissProt record found
Error al procesar el identificador A0A125WDU3: No SwissProt record found
Error al procesar el identificador A0A0G7HBY7: No SwissProt record found
Error al procesar el identificador D3QX74: No SwissProt record found
Error al procesar el identificador U3SW74: No SwissProt record found
Error al procesar el identificador A8J7F6: No SwissProt record found
Error al procesar el identificador F5WVK5: No SwissProt record found
Error al procesar el identificador nan: Failed to find SwissProt entry 'nan'
Error al procesar el identificador E7S2A7: No SwissProt record found
Error al procesar el identificador A0A2H1CGV5: No SwissProt record found
Error al procesar el identificador D6D942: No SwissProt record found


,UniProt IDs,PDB ID,Gene Name,Protein Name,Amino Acid Sequence,Organism Name,Status,Protein Existence,GO BP Terms,GO CC Terms,GO MF Terms
0,Q9CW03,2WD5; 7DG5,"{'Name': 'Smc3', 'Synonyms': ['Bam', 'Bmh', 'C...",Structural maintenance of chromosomes protein 3,MYIKQVIIQGFRSYRDQTIVDPFSSKHNVIVGRNGSGKSNFFYAIQ...,Mus musculus (Mouse).,Reviewed,1,GO:0051301 - cell division; GO:0006281 - DNA r...,GO:0005604 - basement membrane; GO:0000785 - c...,GO:0005524 - ATP binding; GO:0016887 - ATP hyd...
1,Q43155,,"{'Name': 'FdGOGAT {ECO:0000305}', 'ORFNames': ...","Ferredoxin-dependent glutamate synthase, chlor...",MALQSAPKLLYSSPSPSVFSANERRVAFSDFVGLSKKRSRRRRIAG...,Spinacia oleracea (Spinach).,Reviewed,1,GO:0006541 - glutamine metabolic process; GO:0...,GO:0009570 - chloroplast stroma,"GO:0051538 - 3 iron, 4 sulfur cluster binding;..."
2,P30041,1PRX; 5B6M; 5B6N,"{'Name': 'PRDX6', 'Synonyms': ['AOP2', 'KIAA01...",Peroxiredoxin-6,MPGGLLLGDVAPNFEANTTVGRIRFHDFLGDSWGILFSHPRDFTPV...,Homo sapiens (Human).,Reviewed,1,GO:0045454 - cell redox homeostasis; GO:009886...,GO:0035578 - azurophil granule lumen; GO:00057...,GO:0047184 - 1-acylglycerophosphocholine O-acy...
3,P06745,1U0E; 1U0F; 1U0G; 2CVP; 2CXN; 2CXO; 2CXP; 2CXQ...,"{'Name': 'Gpi {ECO:0000303|PubMed:7545951}', '...",Glucose-6-phosphate isomerase {ECO:0000303|Pub...,MAALTRNPQFQKLLEWHRANSANLKLRELFEADPERFNNFSLNLNT...,Mus musculus (Mouse).,Reviewed,1,GO:0061621 - canonical glycolysis; GO:0034101 ...,GO:0060170 - ciliary membrane; GO:0005829 - cy...,GO:0097367 - carbohydrate derivative binding; ...
4,P10809,4PJ1; 6HT7; 6MRC; 6MRD; 7AZP; 7L7S; 8G7J; 8G7K...,"{'Name': 'HSPD1', 'Synonyms': ['HSP60']}","60 kDa heat shock protein, mitochondrial",MLRLPTVFRQMRPVSRVLAPHLTRAYAKDVKFGADARALMLQGVDL...,Homo sapiens (Human).,Reviewed,1,GO:0006458 - 'de novo' protein folding; GO:000...,GO:0009986 - cell surface; GO:0005905 - clathr...,GO:0034186 - apolipoprotein A-I binding; GO:00...


Nota: Algunos identificadores UniProt del conjunto de datos MoonProt están obsoletos o ausentes, lo que impide la recuperación de datos para estas entradas. Estas entradas serán eliminadas en fases posteriores.


## Preparación de los datos para MoonDB
Cargamos y preparamos los identificadores UniProt del conjunto de datos MoonDB para su posterior análisis.

In [ ]:
import pandas as pd

# Cargamos los datos de MoonDB.
data_moondb = pd.read_csv('/content/drive/My Drive/raw_moondb.csv')

# Limipamos los identificadores UniProt de MoonDB.
data_moondb['UniprotKB AC'] = data_moondb['UniprotKB AC'].str.replace('Uniprot: ', '')

uniprot_ids = data_moondb['UniprotKB AC'].tolist()

## Extracción de datos de UniProt para MoonDB
Al igual que con MoonProt, utilizaremos BioPython para extraer datos detallados de cada proteína a partir de su identificador UniProt.

In [ ]:
from Bio import ExPASy, SwissProt

# Inicializamos diccionarios para almacenar la información recopilada para cada proteína.
pdb_ids = {}
gene_names = {}
protein_names = {}
amino_acid_sequences = {}
organism_names = {}
status_info = {}
protein_existence = {}
go_bp_terms = {}
go_cc_terms = {}
go_mf_terms = {}

# Definimos una función para unir datos y manejar valores que podrían no ser cadenas.
def safe_str_join(data, separator="; "):
  if isinstance(data, list):
    return separator.join(str(item) for item in data)
  elif isinstance(data, dict):
    return separator.join(f"{key}: {value}" for key, value in data.items())
  elif data is None:
    return "N/A"
  return str(data)

# Iteramos sobre cada identificador UniProt para realizar consultas y recopilar información.
for uniprot_id in uniprot_ids:
  try:
    handle = ExPASy.get_sprot_raw(uniprot_id)
    record = SwissProt.read(handle)

    # Almacenamos la información específica obtenida de cada registro.
    pdb_entries = [safe_str_join(ref[1]) for ref in record.cross_references if ref[0] == 'PDB']
    pdb_ids[uniprot_id] = safe_str_join(pdb_entries)
    gene_names[uniprot_id] = safe_str_join(record.gene_name, ", ")
    protein_names[uniprot_id] = safe_str_join(record.description.split("=")[1].split(";")[0])
    amino_acid_sequences[uniprot_id] = safe_str_join(record.sequence)
    organism_names[uniprot_id] = safe_str_join(record.organism)
    status_info[uniprot_id] = "Reviewed" if record.data_class == "Reviewed" else "Unreviewed"
    protein_existence[uniprot_id] = safe_str_join(record.protein_existence)

    # Procesamos los términos GO asociados a cada proteína, clasificándolos según su categoría.
    go_terms_processing = lambda ref, code: [f"{r[1]} - {r[2][2:]}" for r in ref if r[0] == "GO" and r[2].startswith(code)]
    go_bp_terms[uniprot_id] = safe_str_join(go_terms_processing(record.cross_references, "P:"))
    go_cc_terms[uniprot_id] = safe_str_join(go_terms_processing(record.cross_references, "C:"))
    go_mf_terms[uniprot_id] = safe_str_join(go_terms_processing(record.cross_references, "F:"))

  except Exception as e:
    print(f"Error al procesar el identificador {uniprot_id}: {e}")
    pdb_ids[uniprot_id] = gene_names[uniprot_id] = protein_names[uniprot_id] = amino_acid_sequences[uniprot_id] = organism_names[uniprot_id] = go_bp_terms[uniprot_id] = go_cc_terms[uniprot_id] = go_mf_terms[uniprot_id] = status_info[uniprot_id] = protein_existence[uniprot_id] = "Error al recuperar datos"

# Actualizamos el DataFrame con los datos recopilados.
data_moondb['PDB ID'] = data_moondb['UniprotKB AC'].map(pdb_ids)
data_moondb['Gene Name'] = data_moondb['UniprotKB AC'].map(gene_names)
data_moondb['Protein Name'] = data_moondb['UniprotKB AC'].map(protein_names)
data_moondb['Amino Acid Sequence'] = data_moondb['UniprotKB AC'].map(amino_acid_sequences)
data_moondb['Organism Name'] = data_moondb['UniprotKB AC'].map(organism_names)
data_moondb['Status'] = data_moondb['UniprotKB AC'].map(status_info)
data_moondb['Protein Existence'] = data_moondb['UniprotKB AC'].map(protein_existence)
data_moondb['GO BP Terms'] = data_moondb['UniprotKB AC'].map(go_bp_terms)
data_moondb['GO CC Terms'] = data_moondb['UniprotKB AC'].map(go_cc_terms)
data_moondb['GO MF Terms'] = data_moondb['UniprotKB AC'].map(go_mf_terms)

data_moondb_final = data_moondb.rename(columns={'UniprotKB AC': 'UniProt IDs'})[
    ['UniProt IDs', 'PDB ID', 'Gene Name', 'Protein Name', 'Amino Acid Sequence', 'Organism Name', 'Status', 'Protein Existence', 'GO BP Terms', 'GO CC Terms', 'GO MF Terms']]

# Visualizamos las primeras filas del DataFrame obtenido.
data_moondb_final.head()

,UniProt IDs,PDB ID,Gene Name,Protein Name,Amino Acid Sequence,Organism Name,Status,Protein Existence,GO BP Terms,GO CC Terms,GO MF Terms
0,Q86VP1,2M7Q; 4BMJ; 4NLH; 4Z4K; 4Z4M; 5AAS; 5YT6; 5Z7G,"{'Name': 'TAX1BP1', 'Synonyms': ['T6BP'], 'ORF...",Tax1-binding protein 1,MTSFQEVPLQTSNFAHVIFQNVAKSYLPNAHLECHYTLTPYIHPHP...,Homo sapiens (Human).,Reviewed,1,GO:0006915 - apoptotic process; GO:0006914 - a...,GO:0005776 - autophagosome; GO:0031410 - cytop...,GO:0019900 - kinase binding; GO:0046872 - meta...
1,Q8WUM4,2OEV; 2OEW; 2OEX; 2OJQ; 2R02; 2R03; 2R05; 2XS1...,{'Name': 'PDCD6IP {ECO:0000312|HGNC:HGNC:8766}...,Programmed cell death 6-interacting protein {E...,MATFISVQLKKTSEVDLAKPLVKFIQQTYPSGGEEQAQYCRAAEEL...,Homo sapiens (Human).,Reviewed,1,GO:0000915 - actomyosin contractile ring assem...,GO:0042641 - actomyosin; GO:0005923 - bicellul...,GO:0048306 - calcium-dependent protein binding...
2,Q6UWE0,,{'Name': 'LRSAM1 {ECO:0000303|PubMed:20865121}...,E3 ubiquitin-protein ligase LRSAM1 {ECO:0000305},MPLFFRKRKPSEEARKRLEYQMCLAKEAGADDILDISKCELSEIPF...,Homo sapiens (Human).,Reviewed,1,GO:0006914 - autophagy; GO:0045806 - negative ...,GO:0005737 - cytoplasm; GO:0005829 - cytosol; ...,GO:0046872 - metal ion binding; GO:0061630 - u...
3,O00499,1MUZ; 1MV0; 1MV3; 2FIC; 2RMY; 2RND; 5I22,"{'Name': 'BIN1', 'Synonyms': ['AMPHL']}",Myc box-dependent-interacting protein 1,MAEMGSKGVTAGKIASNVQKKLTRAQEKVLQKLGKADETKDEQFEQ...,Homo sapiens (Human).,Reviewed,1,GO:0007010 - cytoskeleton organization; GO:000...,GO:0015629 - actin cytoskeleton; GO:0030424 - ...,GO:0051015 - actin filament binding; GO:001982...
4,P06744,1IAT; 1IRI; 1JIQ; 1JLH; 1NUH; 6XUH; 6XUI; 8BBH...,"{'Name': 'GPI {ECO:0000303|PubMed:2387591, ECO...",Glucose-6-phosphate isomerase {ECO:0000303|Pub...,MAALTRDPQFQKLQQWYREHRSELNLRRLFDANKDRFNHFSLTLNT...,Homo sapiens (Human).,Reviewed,1,GO:0005975 - carbohydrate metabolic process; G...,GO:0060170 - ciliary membrane; GO:0005829 - cy...,GO:0097367 - carbohydrate derivative binding; ...


## Limpieza y almacenamiento de los datos
Procedemos a eliminar las filas duplicadas y a filtrar aquellas que carecen de valores válidos en 'GO CC Terms'. Este paso es crucial para descartar las entradas que no pueden ser enriquecidas debido a la falta de identificadores UniProt válidos.


In [ ]:
import pandas as pd

# Eliminamos duplicados.
data_moonprot_filtrado = data_moonprot_final.drop_duplicates(keep='first')
data_moondb_filtrado = data_moondb_final.drop_duplicates(keep='first')

# Filtramos por 'GO CC Terms' válidos.
data_moonprot_filtrado = data_moonprot_filtrado[(data_moonprot_filtrado['GO CC Terms'] != "Error al recuperar datos")]
data_moondb_filtrado = data_moondb_filtrado[(data_moondb_filtrado['GO CC Terms'] != "Error al recuperar datos")]

# Visualizamos las dimensiones de los DataFrames para verificar los cambios.
print(data_moonprot_final.shape, data_moonprot_filtrado.shape)
print(data_moondb_final.shape, data_moondb_filtrado.shape)

# Guardamos los resultados en archivos CSV.
data_moonprot_filtrado.to_csv('/content/drive/My Drive/moonprot_dataset.csv', index=False)
data_moondb_filtrado.to_csv('/content/drive/My Drive/moondb_dataset.csv', index=False)

(473, 11) (451, 11)
(351, 11) (351, 11)


Nota: Después de la limpieza, en MoonProt hemos eliminado 11 filas, mientras que en MoonDB no hemos eliminado ninguna fila.

##  Combinación de los datos

In [ ]:
# Combinamos los conjuntos de datos y eliminamos los duplicados.
moon_dataset = pd.concat([data_moonprot_filtrado, data_moondb_filtrado])
filas_duplicadas_unicas = moon_dataset[moon_dataset.duplicated()]
moon_dataset = moon_dataset.drop_duplicates(keep='first')
print(f"Filas duplicadas únicas: {len(filas_duplicadas_unicas)}")
print(f"Dimensiones sin duplicados: {moon_dataset.shape}")

Filas duplicadas únicas: 33
Dimensiones sin duplicados: (769, 11)


## Almacenamiento del conjunto de datos final
Finalmente, guardamos el conjunto de datos combinado y limpio en un archivo CSV, y visualizamos las primeras entradas para confirmar la calidad de los datos.

In [ ]:
moon_dataset.to_csv('/content/drive/My Drive/moon_dataset.csv', index=False)
print(moon_dataset.head())

  UniProt IDs                                             PDB ID  \
0      Q9CW03                                         2WD5; 7DG5   
1      Q43155                                                      
2      P30041                                   1PRX; 5B6M; 5B6N   
3      P06745  1U0E; 1U0F; 1U0G; 2CVP; 2CXN; 2CXO; 2CXP; 2CXQ...   
4      P10809  4PJ1; 6HT7; 6MRC; 6MRD; 7AZP; 7L7S; 8G7J; 8G7K...   

                                           Gene Name  \
0  {'Name': 'Smc3', 'Synonyms': ['Bam', 'Bmh', 'C...   
1  {'Name': 'FdGOGAT {ECO:0000305}', 'ORFNames': ...   
2  {'Name': 'PRDX6', 'Synonyms': ['AOP2', 'KIAA01...   
3  {'Name': 'Gpi {ECO:0000303|PubMed:7545951}', '...   
4           {'Name': 'HSPD1', 'Synonyms': ['HSP60']}   

                                        Protein Name  \
0    Structural maintenance of chromosomes protein 3   
1  Ferredoxin-dependent glutamate synthase, chlor...   
2                                    Peroxiredoxin-6   
3  Glucose-6-phosphate isomera